In [1]:
!pip freeze | grep tensorflow

tensorflow==2.1.0
tensorflow-datasets==1.3.0
tensorflow-estimator==2.1.0
tensorflow-metadata==0.15.0


# 提交记录

## phase2
### 日期:2020-04-19 17:06:44排名: 无
user-item和item_item融合 想着会有提升，有些失望！

* score:0.1343
* hitrate_50_full:0.3238
* ndcg_50_full:0.1343
* hitrate_50_half:0.2429
* ndcg_50_half:0.1013

### 日期:2020-04-19 16:45:28排名: 无
user-item  单个user_item的得分还是很满意的

score:0.1295
hitrate_50_full:0.3161
ndcg_50_full:0.1295
hitrate_50_half:0.2340
ndcg_50_half:0.0932
### 日期:2020-04-19 00:25:12排名: 无  对user点击item按时间做指数加权处理

score:0.1275
hitrate_50_full:0.3071
ndcg_50_full:0.1275
hitrate_50_half:0.2171
ndcg_50_half:0.0952

### 日期:2020-04-18 20:54:35排名: 无  对user点击item按时间做线性加权处理

* score:0.1353
* hitrate_50_full:0.3291
* ndcg_50_full:0.1353
* hitrate_50_half:0.2454
* ndcg_50_half:0.1010

### 日期:2020-04-18 18:48:13排名: 无 phase2 use_iif=False

score:0.1195
hitrate_50_full:0.3001
ndcg_50_full:0.1195
hitrate_50_half:0.2228
ndcg_50_half:0.0857

### 日期:2020-04-18 18:41:10排名: 无 phase2

score:0.1264
hitrate_50_full:0.3139
ndcg_50_full:0.1264
hitrate_50_half:0.2331
ndcg_50_half:0.0934
## phase1
### 日期:2020-04-17 16:07:36排名: 无 phase1

score:0.0841
hitrate_50_full:0.2068
ndcg_50_full:0.0841
hitrate_50_half:0.1581
ndcg_50_half:0.0632
改进：
用所有的数据预测phasen
初始版本是用phase0-n的数据预测phasen，而没有用到后面的数据

### 日期:2020-04-17 15:22:28排名: 无
修改了一行bug
for j, wij in sorted(sim_item_corr[i].items(), reverse=True)[:top_k]:

score:0.0787
hitrate_50_full:0.1996
ndcg_50_full:0.0787
hitrate_50_half:0.1383
ndcg_50_half:0.0556

### 高手开源baseline
https://tianchi.aliyun.com/forum/postDetail?postId=103530
日期:2020-04-17 10:42:13

score:0.0707
hitrate_50_full:0.1784
ndcg_50_full:0.0707
hitrate_50_half:0.1405
ndcg_50_half:0.0558
### 日期:2020-04-13 18:37:27排名: 无  这么惨，因为有一个很明显的bug。。。

score:0.0006
hitrate_50_full:0.0030
ndcg_50_full:0.0006
hitrate_50_half:0.0011
ndcg_50_half:0.0002

# baseline0_itemCF

In [78]:
!python baseline0_itemCF.py

gen train_data_matrix begin...
cal cosine_similarity begin...
cal cosine_similarity end
100%|████████████████████████████████████| 27195/27195 [00:45<00:00, 600.60it/s]
935.4970018863678s
complete!!!


# baseline1_itemCF

In [314]:
!python baseline1_itemCF.py

phase: 0
phase: 1
phase: 2
phase: 3
100%|███████████████████████████████████████| 6754/6754 [02:01<00:00, 55.63it/s]
151.77655220031738s
complete!!!


In [310]:
!python baseline1_itemCF.py

phase: 0
phase: 1
phase: 2
phase: 3
100%|███████████████████████████████████████| 6754/6754 [01:52<00:00, 60.09it/s]
140.00655221939087s
complete!!!


# 思路总结
## baseline0_itemCF
通过建立 user-item矩阵，通过用户的点击行为得出item的相似性，如item1和item2都只被u1和u2各点击过1次，那么item1和item2相似度为1

推荐过程:
已知u1点击了i1，那么拿到i1最相近的前50个item推荐


## baseline1_itemCF
通过建立 item-item共现矩阵，通过item1和item2被同一user同时点击来计算item的相似性，如u1点击过i1和i2，则对i1和i2共现次数贡献1，如果u2也点击过i1和i2，则对i1和i2共现次数贡献+1

推荐过程：
并没有简单的将user最近一次购买的item共现系数最高的前50推荐，而是考虑了user历史购买过的所有item，把与这些item共现系数最高的topk=500个item都找出来，然后取出其中 user没有购买过的 共现系数最高的前50个做推荐，如果不足50个，则用所有点击中最高的前50item补足



## 问题：
1. 没有考虑user，即u1点击i1和u2点击i1的推荐结果一样
baseline0_itemCF
2. 没有考虑user的历史点击行为，即u1在点击i1之前点击了i2或者i3的推荐结果一样
baseline0_itemCF
3. 没有考虑user自身特征
baseline0_itemCF
baseline1_itemCF
4. 没有考虑item自身特征
baseline0_itemCF
baseline1_itemCF
5. 没有考虑user对item的点击次数影响，即u1对i1点击过1次、对i2点击过1次和u2对i1点击过100次、对i2点击过100次，在计算i1和i2的共现系数时效果相同
baseline1_itemCF


## 待改进
### baseline0_itemCF
建立 user-item矩阵，
点击过程需要做平滑，如u1对i1点击1次，和u1对i1点击10000次要做平滑，如在计算完点击次数后，取log(cnt+1)，这样，点击0次，为0，点击>0次，结果大于0

推荐过程可以参照baseline1_itemCF的做改进
取出历史购买过的所有item，然后再在里面取出前50做推荐，而不是只考虑最近点击过的一个item

### baseline1_itemCF
推荐过程 改进
1. item添加权重 第一次点击的item是最后一次点击item权重的0.5 done  
分数明显改善！  
如一个用户共点击过100个item，那么最开始点击的item1，肯定要比最近点击的item100对推荐的影响要小，因此item1的相似的topk个物品的权重要小于item100的相似的topk个物品的权重  
本来想着可以用time作为权重，但是它们的差异实在太小了！  
算了，自己造一个把，最开始点击的item的权重是0.5，最近一次点击的item权重取1，如果item被user点击了多次，则去最近点击的那次
2. item权重调优  优先级调低，后面再做！  
相同time的权重应该相同，看了一下，应该是没有相同的time。。。

权重应该和time的值有关系，而不是排序。如，有2个值的time序列，[5, 6], [5, 10], [5, 20], 目前的处理方式，第一个的权重都是第二个权重的0.5，但显然[5, 6]中第一个值权重应该大于[5, 10]中第一个值权重应该大于[5, 20]中第一个值权重。

线性衰减，也可以考虑指数衰减，相差越大 衰减的越快！ 指数衰减貌似更接近实际情况
指数衰减不太试了一下，效果不好，可能区间太大了。


In [ ]:
#!/usr/bin/env python  
# -*- coding:utf-8 -*-  

import numpy as np
import pandas as pd  
from tqdm import tqdm  
from collections import defaultdict  
import math  
from sklearn.metrics.pairwise import cosine_similarity
  
def get_sim_item(df, user_col, item_col, use_iif=False, mode='item_item'): 
    """
    use_iif:
        True: 把用户点击过的item总数考虑进去，
            如果用户1和用户2都点击了item[1, 2]，但是用户1一共就点击过10个item，而用户2一共点击过10000个item
            那么在计算item[1, 2]的共现系数时，用户1肯定比用户2的价值大
    """
    user_item_dict = None
    if mode == 'item_item':
        user_item_ = df.groupby(user_col)[item_col].agg(set).reset_index()  
        user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))  

        sim_item = {}  # sim_item[i][j] 表示item_i和item_j共现的次数
        item_cnt = defaultdict(int)  # 统计某个item有多少个user购买过
        for user, items in tqdm(user_item_dict.items()):  
            for i in items:  
                item_cnt[i] += 1  
                sim_item.setdefault(i, {})  
                for relate_item in items:  
                    if i == relate_item:  
                        continue  
                    sim_item[i].setdefault(relate_item, 0)  
                    if not use_iif:  
                        sim_item[i][relate_item] += 1  
                    else:  
                        sim_item[i][relate_item] += 1 / math.log(1 + len(items))  

        sim_item_corr = sim_item.copy()  
        for i, related_items in tqdm(sim_item.items()):  
            for j, cij in related_items.items():  
                # item_i和item_j共现的次数/各自出现的次数之积
                sim_item_corr[i][j] = cij/math.sqrt(item_cnt[i]*item_cnt[j])  
    elif mode == 'user_item':
        click_df = df
        click_df = click_df.drop_duplicates()

        n_users = click_df.user_id.nunique()

        n_items = click_df.item_id.nunique()

        print('gen train_data_matrix begin...')
        train_data_matrix = np.zeros((n_users, n_items))
        user_id_dict, item_id_dict = dict(), dict()
        user_id_dict_inv, item_id_dict_inv = dict(), dict()
        u_cnt, i_cnt = 0, 0
        for line in click_df.itertuples():
            if line.user_id not in user_id_dict:
                user_id_dict[line.user_id] = u_cnt
                user_id_dict_inv[u_cnt] = line.user_id
                u_cnt += 1
            if line.item_id not in item_id_dict:
                item_id_dict[line.item_id] = i_cnt
                item_id_dict_inv[i_cnt] = line.item_id
                i_cnt += 1
            train_data_matrix[user_id_dict[line.user_id], item_id_dict[line.item_id]] += 1
        train_data_matrix = np.log(train_data_matrix + 1)
        print('cal cosine_similarity begin...')  
        #所有test中出现的user购买过的item_id 改为click_test中user购买的item_id
#         all_test_item_id = df.loc[df.user_id.isin(click_test.user_id.unique().tolist()), 'item_id'].unique().tolist()
        all_test_item_id = click_test.item_id.unique().tolist()
        # 把item_id转换为matrix index
        tt = [item_id_dict[i] for i in all_test_item_id]    
        print(len(tt))
        t = cosine_similarity(train_data_matrix.T, train_data_matrix[:, tt].T)
#         xb = train_data_matrix.T
#         xq = train_data_matrix[:, tt].T
#         import faiss                   # make faiss available
# #         faiss.normalize_L2(xb)
# #         faiss.normalize_L2(xq)
#         index = faiss.IndexFlatIP(n_users)   # build the index
#         print(index.is_trained)
#         index.add(xb)                  # add vectors to the index
#         print(index.ntotal)
#         D, I = index.search(xq, 500)
        print('cal cosine_similarity end')
        dft = pd.DataFrame(t)
        dft.to_pickle('dft.pkl')
#         dft = pd.read_pickle('dft.pkl')
        print(dft.shape)
        sim_item_corr = dict()
#         for i in tqdm(range(I.shape[0])):
#             sim_item_corr[item_id_dict_inv[tt[i]]] = {item_id_dict_inv[v]: D[i, ii] for ii, v in enumerate(I[i, 1:])}
        for i in tqdm(range(dft.shape[1])):
            sim_item_corr[item_id_dict_inv[tt[i]]] = {item_id_dict_inv[ii]: v 
                                                      for ii, v in dft.iloc[:, i].sort_values(ascending=False).iloc[:501].items()}
    return sim_item_corr, user_item_dict  
  

def recommend(sim_item_corr, user_item_dict, user_id, top_k, item_num, weight_mode='linear'):  
    """
    向user_id推荐item_num个该user_id没有买过的item
    """
    rank = {}  
    # 该user_id购买过的items
    dft = click_test[click_test.user_id == user_id].sort_values('time').drop_duplicates('item_id', keep='last')
    if weight_mode=='linear':
        dft['t'] = range(dft.shape[0], dft.shape[0] * 2)
        dft['t'] = dft['t'] - dft.shape[0] * 3//4
    elif weight_mode=='exp':
        dft['t'] = ((dft['time'] - 0.9837) * 10000).map(math.exp) # 最大值是最小值的15倍左右，score:0.1275
        
    interacted_items = dft['item_id'].tolist()    
    weights = dft['t'].tolist()
#     print(dft.shape[0], dft.head())
    # 遍历该user购买过的items，
    cnt = 0
    for i in interacted_items:  
        # 遍历该item共现最高的top_k个item，把其中用户没有买过的加入推荐列表
#         for j, wij in sorted(sim_item_corr[i].items(), reverse=True)[:top_k]:  
        for j, wij in sorted(sim_item_corr[i].items(), key=lambda d: d[1], reverse=True)[:top_k]:  
            if j not in interacted_items:  
                rank.setdefault(j, 0)  
                rank[j] += wij * weights[cnt]
        cnt += 1
    return sorted(rank.items(), key=lambda d: d[1], reverse=True)[:item_num]  
  
def get_predict(df, pred_col, top_fill):  
    """
    fill user to 50 items
    逻辑就是如果推荐给用户的items少于50个，就用点击数最高的item补足
    """
    top_fill = [int(t) for t in top_fill.split(',')]  
    scores = [-1 * i for i in range(1, len(top_fill) + 1)]  
    ids = list(df['user_id'].unique())  
    fill_df = pd.DataFrame(ids * len(top_fill), columns=['user_id'])  
    fill_df.sort_values('user_id', inplace=True)  
    fill_df['item_id'] = top_fill * len(ids)  
    fill_df[pred_col] = scores * len(ids)  
    df = df.append(fill_df)  
    df.sort_values(pred_col, ascending=False, inplace=True)  
    df = df.drop_duplicates(subset=['user_id', 'item_id'], keep='first')  
    df['rank'] = df.groupby('user_id')[pred_col].rank(method='first', ascending=False)  
    df = df[df['rank'] <= 50]  
    df = df.groupby('user_id')['item_id'].apply(lambda x: ','.join([str(i) for i in x])).str.split(',', expand=True).reset_index()  
    return df  

In [2]:
def load_data(now_phase):
#     now_phase = 2  
    train_path = './data_origin/underexpose_train'  
    test_path = './data_origin/underexpose_test'  
    recom_item = []  

    whole_click = pd.DataFrame()  
    click_train = pd.DataFrame()   
    click_test = pd.DataFrame()  
    test_qtime = pd.DataFrame()  
    for c in range(now_phase + 1):  
        print('phase:', c)  
        click_train1 = pd.read_csv(train_path + '/underexpose_train_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'])  
        click_test1 = pd.read_csv(test_path + '/underexpose_test_click-{}/underexpose_test_click-{}.csv'.format(c, c), header=None,  names=['user_id', 'item_id', 'time'])  
        test_qtime1 = pd.read_csv(test_path + '/underexpose_test_click-{}/underexpose_test_qtime-{}.csv'.format(c, c), header=None,  names=['user_id','query_time'])  

        click_train = click_train.append(click_train1) 
    #     all_click = click_train.append(click_test1)  
        click_test = click_test.append(click_test1) 
        test_qtime = test_qtime.append(test_qtime1) 

#     # 去掉 train中time>query_time的数据    
#     click_train = pd.merge(click_train, test_qtime, how='left').fillna(10)  
#     click_train = click_train[click_train.time <= click_train.query_time]
#     del click_train['query_time']
    whole_click = click_train.append(click_test)  
    whole_click = whole_click.drop_duplicates()
    return whole_click, click_train, click_test, test_qtime


In [3]:
now_phase = 3
whole_click, click_train, click_test, test_qtime = load_data(now_phase)
# item_sim_list, user_item = get_sim_item(whole_click, 'user_id', 'item_id', use_iif=True, mode='item_item')
item_sim_list, user_item = get_sim_item(whole_click, 'user_id', 'item_id', use_iif=True, mode='user_item')   


recom_item = [] 
for i in tqdm(click_test['user_id'].unique()):  
    rank_item = recommend(item_sim_list, user_item, i, 500, 50)  
    for j in rank_item:  
        recom_item.append([i, j[0], j[1]])  
# find most popular items  
top50_click = whole_click['item_id'].value_counts().index[:50].values  
top50_click = ','.join([str(i) for i in top50_click])  

recom_df = pd.DataFrame(recom_item, columns=['user_id', 'item_id', 'sim'])  
result = get_predict(recom_df, 'sim', top50_click)  

result.to_csv('/Users/luoyonggui/Downloads/baseline1_itemcf2.csv', index=False, header=None)

phase: 0
phase: 1
phase: 2
phase: 3
gen train_data_matrix begin...
cal cosine_similarity begin...
42800
cal cosine_similarity end


  0%|          | 0/42800 [00:00<?, ?it/s]

(71904, 42800)


100%|██████████| 6754/6754 [01:03<00:00, 106.67it/s]


# 重新来

In [2]:
from datetime import datetime
def get_sim_item(df, user_col, item_col, use_iif=False): 
    """
    use_iif:
        True: 把用户点击过的item总数考虑进去，
            如果用户1和用户2都点击了item[1, 2]，但是用户1一共就点击过10个item，而用户2一共点击过10000个item
            那么在计算item[1, 2]的共现系数时，用户1肯定比用户2的价值大
    """
    click_df = df
    click_df = click_df.drop_duplicates()

    n_users = click_df.user_id.nunique()

    n_items = click_df.item_id.nunique()

    print('gen train_data_matrix begin...')
#     train_data_matrix = np.zeros((n_users, n_items))
    import scipy.sparse as sp
    train_data_matrix = sp.lil_matrix((n_users, n_items))
    user_id_dict, item_id_dict = dict(), dict()
    user_id_dict_inv, item_id_dict_inv = dict(), dict()
    u_cnt, i_cnt = 0, 0
    for line in click_df.itertuples():
        if line.user_id not in user_id_dict:
            user_id_dict[line.user_id] = u_cnt
            user_id_dict_inv[u_cnt] = line.user_id
            u_cnt += 1
        if line.item_id not in item_id_dict:
            item_id_dict[line.item_id] = i_cnt
            item_id_dict_inv[i_cnt] = line.item_id
            i_cnt += 1
        train_data_matrix[user_id_dict[line.user_id], item_id_dict[line.item_id]] += 1
    train_data_matrix = train_data_matrix.tocsc()
    train_data_matrix.data = np.log(train_data_matrix.data + 1)
#     train_data_matrix = np.log(train_data_matrix + 1) + np.random.RandomState(14).uniform(size=(n_users, n_items))*0.001
    print(f'cal cosine_similarity begin...{datetime.now()}')  
    #所有test中出现的user购买过的item_id 改为click_test中user购买的item_id
    all_test_item_id = df.loc[df.user_id.isin(click_test.user_id.unique().tolist()), 'item_id'].unique().tolist()
#     all_test_item_id = click_test.item_id.unique().tolist()
    # 把item_id转换为matrix index
    tt = [item_id_dict[i] for i in all_test_item_id]    
    print(len(tt))
    t = cosine_similarity(train_data_matrix.T, train_data_matrix[:, tt].T)
    dft = pd.DataFrame(t)
#     dft.to_pickle('dft.pkl')
#   dft = pd.read_pickle('dft.pkl')
#     xb = train_data_matrix.T
#     xq = train_data_matrix[:, tt].T
#     import faiss                   # make faiss available
#     faiss.normalize_L2(xb)
#     faiss.normalize_L2(xq)
#     index = faiss.IndexFlatIP(n_users)   # build the index
#     print(index.is_trained)
#     index.add(xb)                  # add vectors to the index
#     print(index.ntotal)
#     D, I = index.search(xq, 500)
    print(f'cal cosine_similarity end..{datetime.now()}')
#     print(dft.shape)
    sim_item_corr = dict()
#     for i in tqdm(range(I.shape[0])):
#         sim_item_corr[item_id_dict_inv[tt[i]]] = {item_id_dict_inv[v]: D[i, ii] for ii, v in enumerate(I[i, 1:])}
    for i in tqdm(range(dft.shape[1])):
        sim_item_corr[item_id_dict_inv[tt[i]]] = {item_id_dict_inv[ii]: v 
                                                  for ii, v in dft.iloc[:, i].sort_values(ascending=False).iloc[:501].items()}
    return sim_item_corr, user_item_dict  
  

In [15]:
def get_sim_item(df, user_col, item_col, use_iif=False):  
    user_item_ = df.groupby(user_col)[item_col].agg(set).reset_index()  
    user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))  
    
    item_user_ = df.groupby(item_col)[user_col].agg(set).reset_index()  
    item_user_dict = dict(zip(item_user_[item_col], item_user_[user_col]))    

    item_cnt = defaultdict(int)  
    for user, items in tqdm(user_item_dict.items()):  
        for i in items:  
            item_cnt[i] += 1  

    sim_item = {}

    for item, users in tqdm(item_user_dict.items()):
    
        sim_item.setdefault(item, {}) 
    
        for u in users:
        
            tmp_len = len(user_item_dict[u])
        
            for relate_item in user_item_dict[u]:
                sim_item[item].setdefault(relate_item, 0)
                sim_item[item][relate_item] += 1/ (math.log(len(users)+1) * math.log(tmp_len+1))
            

            
            
    return sim_item, user_item_dict  

In [1]:
from myutils import *

import pandas as pd  
from tqdm import tqdm  
from collections import defaultdict  
import math  
from sklearn.metrics.pairwise import cosine_similarity
  
def get_sim_item(df, user_col, item_col, use_iif=False):  
    user_item_ = df.groupby(user_col)[item_col].agg(set).reset_index()  
    user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))  
  
    sim_item = {}  
    item_cnt = defaultdict(int)  
    for user, items in tqdm(user_item_dict.items()):  
        for i in items:  
            item_cnt[i] += 1  
            sim_item.setdefault(i, {})  
            for relate_item in items:  
                if i == relate_item:  
                    continue  
                sim_item[i].setdefault(relate_item, 0)  
                if not use_iif:  
                    sim_item[i][relate_item] += 1  
                else:  
                    sim_item[i][relate_item] += 1 / math.log(1 + len(items))  
    sim_item_corr = sim_item.copy()  
    for i, related_items in tqdm(sim_item.items()):  
        for j, cij in related_items.items():  
#             sim_item_corr[i][j] = cij/math.sqrt(item_cnt[i]*item_cnt[j]) 
            sim_item_corr[i][j] = cij/(np.log(item_cnt[i]+1) * np.log(item_cnt[j]+1)) 
#             sim_item_corr[i][j] = cij/np.log((item_cnt[i]+1)*(item_cnt[j]+1)) 
  
    return sim_item_corr, user_item_dict  


def recommend(whole_click_train_, sim_item_corr, user_item_dict, user_id, top_k, item_num, weight_mode='linear'):  
    rank = {}  
    # 该user_id购买过的items
    dft = whole_click_train_[whole_click_train_.user_id == user_id].sort_values('time').drop_duplicates('item_id', keep='last')
    if weight_mode=='linear':
        dft['t'] = range(dft.shape[0], dft.shape[0] * 2)
        dft['t'] = dft['t'] - dft.shape[0] // 2 - dft.shape[0] // 4
#         dft['t'] = dft['t'] - dft.shape[0] // 2 - dft.shape[0] // 4 - dft.shape[0] // 8
#         dft['t'] = range(1, dft.shape[0] + 1)
#         dft['t'] = range(0, dft.shape[0])
    elif weight_mode=='exp':
        dft['t'] = ((dft['time'] - 0.9837) * 10000).map(math.exp) # 最大值是最小值的15倍左右，score:0.1275
        
    interacted_items = dft['item_id'].tolist()    
    weights = dft['t'].tolist()
    cnt = 0
    for i in interacted_items:  
        for j, wij in sorted(sim_item_corr[i].items(), key=lambda d: d[1], reverse=True)[0:top_k]:  
            if j not in interacted_items:  
                rank.setdefault(j, 0)  
                rank[j] += wij  * weights[cnt]
        cnt += 1
    return sorted(rank.items(), key=lambda d: d[1], reverse=True)[:item_num]  
  

# fill user to 50 items  
def get_predict(df, pred_col, top_fill):  
    top_fill = [int(t) for t in top_fill.split(',')]  
    scores = [-1 * i for i in range(1, len(top_fill) + 1)]  
    ids = list(df['user_id'].unique())  
    fill_df = pd.DataFrame(ids * len(top_fill), columns=['user_id'])  
    fill_df.sort_values('user_id', inplace=True)  
    fill_df['item_id'] = top_fill * len(ids)  
    fill_df[pred_col] = scores * len(ids)  
    df = df.append(fill_df)  
    df.sort_values(pred_col, ascending=False, inplace=True)  
    df = df.drop_duplicates(subset=['user_id', 'item_id'], keep='first')  
    df['rank'] = df.groupby('user_id')[pred_col].rank(method='first', ascending=False)  
    df = df[df['rank'] <= 50]  
    df = df.groupby('user_id')['item_id'].apply(lambda x: ','.join([str(i) for i in x])).str.split(',', expand=True).reset_index()  
    return df  

In [6]:
def main(whole_click_train_, answers_, test_qtime_):
    import time
    t0 = time.time()
    recom_item = []  
    item_sim_list, user_item = get_sim_item(whole_click_train_, 'user_id', 'item_id', use_iif=True)  

    for i in tqdm(test_qtime_['user_id'].unique()):  
        rank_item = recommend(whole_click_train_, item_sim_list, user_item, i, 500, 50)  
        for j in rank_item:  
            recom_item.append([i, j[0], j[1]])  
    # find most popular items  
    top50_click = whole_click_train_['item_id'].value_counts().index[:50].values  
    top50_click = ','.join([str(i) for i in top50_click])  

    recom_df = pd.DataFrame(recom_item, columns=['user_id', 'item_id', 'sim'])  
    result = get_predict(recom_df, 'sim', top50_click)  
    
    result1 = result.set_index('user_id')
    result1 = result1.applymap(int)
    result1['p'] = result1.values.tolist()
    print(evaluate_each_phase(result1['p'].to_dict(), answers_))
    
    t1 = time.time()
    print(f'{t1-t0}s')
    return result, item_sim_list

In [7]:
now_phase = 4
whole_click_train, click_test_val, test_qtime, all_click_df = load_click_data(now_phase)

val = click_test_val[['user_id', 'item_id', 'item_deg']].set_index('user_id')
val['p'] = val.values.tolist()
answers = val['p'].to_dict()

phase: 0
phase: 1
phase: 2
phase: 3
phase: 4


In [9]:
result, item_sim_list = main(whole_click_train, answers, test_qtime)

100%|██████████| 8462/8462 [04:32<00:00, 31.08it/s]


[0.08650437 0.03250136 0.0629975  0.02177384]
390.4417133331299s


In [10]:
result = main(whole_click_train.append(click_test_val), answers, test_qtime)
result[0].to_csv('/Users/luoyonggui/Downloads/baseline1_itemcf24.csv', index=False, header=None)

100%|██████████| 8462/8462 [04:31<00:00, 31.14it/s]


[0. 0. 0. 0.]
396.76055121421814s


In [ ]:
phase4
[0.08650437 0.03250136 0.0629975  0.02177384]
0.6090 0.2496 0.3724 0.1374  --online

In [146]:
# 0 原始的qyxs版本
# array([0.05567071, 0.02003306, 0.05627462, 0.02107223], dtype=float32)
# 0.1 修改了其中的排序bug
# array([0.06944034, 0.02491091, 0.05993247, 0.02189971], dtype=float32)
# 0.1.1 get_sim_item1 sim_item_corr[i][j] = cij / (np.log(item_cnt[i]+1) * np.log(item_cnt[j]+1))
# array([0.07817589, 0.02840868, 0.05374226, 0.01814048], dtype=float32)
# 0.1.1.1 添加线性时间衰减 0.5
# [0.08779982, 0.03312078, 0.06077659, 0.02157981], dtype=float32)
# 0.1.1.2 添加线性时间衰减 0.5 //2 //4
# [0.09076103 0.03535827 0.06499719 0.02348245]
# 0.4925 0.2029 0.3096 0.1181  --------------------online
# 0.1.1.3 添加线性时间衰减 0.5 //2 //4 //8
# [0.09135327 0.03565276 0.06527856 0.02362418]
# 0.1.1.4 添加线性时间衰减  range(1, dft.shape[0] + 1)
# [0.09327806 0.03668222 0.0658413  0.02404739]
# 0.4865 0.2010 0.3061 0.1167   --------------------online
# 0.1.1.5 添加线性时间衰减  range(0, dft.shape[0])
# [0.09150133 0.03644358 0.06443444 0.02380092]
# 0.1.1.2.1  get_sim_item1 sim_item_corr[i][j] = cij/np.log((item_cnt[i]+1)*(item_cnt[j]+1)) 
# [0.08794788 0.03424193 0.04895892 0.01559871]
# 0.1.2 添加线性时间衰减 //2 //4
# array([0.08424637, 0.03159376, 0.07090602, 0.0267413 ], dtype=float32)
# 0.2 双向网络
# 0.2005 0.4871 0.2005 0.2059 0.0678
# 0.3 改进版，没有修改排序bug
# array([0.03050044, 0.01043385, 0.03685988, 0.01230763], dtype=float32)
# 改进版，效果有点出乎意料啊。。。
# array([0.03642286, 0.01217174, 0.03629713, 0.01195004], dtype=float32)
# 感觉recommend的减少权重有点过 0.7**loc，先去掉看看，效果稍好
# array([0.03997631, 0.0140847 , 0.01660101, 0.00548362], dtype=float32)
# 修正逆向位置权重 0.5 * (0.9**(loc1-loc2-1))
# array([0.06114895, 0.02198378, 0.03292065, 0.01053025], dtype=float32)
# 修正逆向位置权重 0.8 * (0.9**(loc1-loc2-1)) 
# array([0.06499852, 0.02380453, 0.03376477, 0.01079678], dtype=float32)
# get_sim_item1 sim_item_corr[i][j] = cij / ((item_cnt[i] * item_cnt[j]))
# array([0.03523838, 0.01321244, 0.05486776, 0.02201659], dtype=float32)
# get_sim_item1 sim_item_corr[i][j] = cij / (np.log(item_cnt[i]+1) * np.log(item_cnt[j]+1))
# array([0.06440628, 0.02371753, 0.04361283, 0.01563565], dtype=float32)
evaluate_each_phase(result1['p'].to_dict(), answers)

array([0.08424637, 0.03159376, 0.07090602, 0.0267413 ], dtype=float32)

# 重构

## load data

In [13]:
import scipy.sparse as sp

In [11]:
now_phase = 4
whole_click_train, click_test_val, test_qtime, all_click_df = load_click_data(now_phase)

val = click_test_val[['user_id', 'item_id', 'item_deg']].set_index('user_id')
val['p'] = val.values.tolist()
answers = val['p'].to_dict()

phase: 0
phase: 1
phase: 2
phase: 3
phase: 4


In [15]:
user_col = 'user_id'
item_col = 'item_id'

In [44]:
def get_item_item_matrix(click_df, user_col, item_col, use_iif=False):
    click_df_ = click_df.copy()
    click_df_ = click_df_.sort_values('time')
    n_items = click_df_.item_id.nunique()
    print(f'n_items: {n_items}')
    user_item_ = click_df_.groupby(user_col)[item_col, 'time'].agg(list).reset_index() 
    click_time_std = click_df_.time.describe().loc['std']
    item_cnt = click_df_.groupby(item_col)['time'].count()
    train_data_matrix = sp.lil_matrix((n_items, n_items))
    print(train_data_matrix.shape)
    cnt = 0
    item_id_dict, item_id_dict_inv = dict(), dict()
    for line in tqdm(user_item_.itertuples()):
        for i, item in enumerate(line.item_id):
    #         print(i, item)
    #         break
            if item not in item_id_dict:
                item_id_dict[item] = cnt
                item_id_dict_inv[cnt] = item
                cnt += 1
            tmp = 0
            for j, relate_item in enumerate(line.item_id):
                if item == relate_item: continue
                if relate_item not in item_id_dict:
                    item_id_dict[relate_item] = cnt
                    item_id_dict_inv[cnt] = relate_item
                    cnt += 1
                tmp += 1
                # 如果用户a共点击了n个items， 那n越大，共现的价值越小  
                tmp /= math.log(1 + len(line.item_id)) 
                # items点击的time间隔越远，共现价值越小
                # items的先后顺序，先a后b，则b的价值高，先b后a，则b的价值低
                alpha = 1
                if line.time[i] < line.time[j]:
                    alpha = 0.9
                tmp = alpha * 1/math.exp(math.fabs(line.time[i]-line.time[j])/click_time_std)
                # items一共被点击的次数，被点击次数越多，价值越低
    #             print(item_cnt.loc[item])
                tmp /= (math.log(item_cnt.loc[item]+1)*math.log(item_cnt.loc[relate_item]+1))
    #             print(item_id_dict[item], item_id_dict[relate_item], tmp)
                train_data_matrix[item_id_dict[item],item_id_dict[relate_item]] += tmp
    pd.Series(item_id_dict).to_pickle('data_gen/item_id_series.pkl')
    pd.Series(item_id_dict_inv).to_pickle('data_gen/item_id_series_inv.pkl')
    sp.save_npz('data_gen/item_item_train_phase4.npz', train_data_matrix.tocsc())
    return train_data_matrix, item_id_dict, item_id_dict_inv

27768it [34:59, 13.23it/s]


In [49]:
sp.save_npz('data_gen/item_item_train_phase4.npz', train_data_matrix_)

In [63]:
def recommend(whole_click_train_, train_data_matrix1, item_id_dict, item_id_dict_inv, user_id, top_k, item_num, weight_mode='linear'):  
    rank = {}  
    # 该user_id购买过的items
    dft = whole_click_train_[whole_click_train_.user_id == user_id].sort_values('time').drop_duplicates('item_id', keep='last')
    if weight_mode=='linear':
        dft['t'] = range(dft.shape[0], dft.shape[0] * 2)
#         dft['t'] = dft['t'] - dft.shape[0] // 2 - dft.shape[0] // 4
#         dft['t'] = dft['t'] - dft.shape[0] // 2 - dft.shape[0] // 4 - dft.shape[0] // 8
#         dft['t'] = range(1, dft.shape[0] + 1)
#         dft['t'] = range(0, dft.shape[0])
    elif weight_mode=='exp':
        dft['t'] = ((dft['time'] - 0.9837) * 10000).map(math.exp) # 最大值是最小值的15倍左右，score:0.1275
        
    interacted_items = dft['item_id'].tolist()    
    weights = dft['t'].tolist()
    cnt = 0
    for i in interacted_items:  
        st = pd.Series(train_data_matrix1.data[item_id_dict[i]], index=train_data_matrix1.rows[item_id_dict[i]])
        for j, wij in st.sort_values().iloc[0:top_k].items():  
            if item_id_dict_inv[j] not in interacted_items:  
                rank.setdefault(item_id_dict_inv[j], 0)  
                rank[item_id_dict_inv[j]] += wij  * weights[cnt]
        cnt += 1
    return sorted(rank.items(), key=lambda d: d[1], reverse=True)[:item_num]  
  

# fill user to 50 items  
def get_predict(df, pred_col, top_fill):  
    top_fill = [int(t) for t in top_fill.split(',')]  
    scores = [-1 * i for i in range(1, len(top_fill) + 1)]  
    ids = list(df['user_id'].unique())  
    fill_df = pd.DataFrame(ids * len(top_fill), columns=['user_id'])  
    fill_df.sort_values('user_id', inplace=True)  
    fill_df['item_id'] = top_fill * len(ids)  
    fill_df[pred_col] = scores * len(ids)  
    df = df.append(fill_df)  
    df.sort_values(pred_col, ascending=False, inplace=True)  
    df = df.drop_duplicates(subset=['user_id', 'item_id'], keep='first')  
    df['rank'] = df.groupby('user_id')[pred_col].rank(method='first', ascending=False)  
    df = df[df['rank'] <= 50]  
    df = df.groupby('user_id')['item_id'].apply(lambda x: ','.join([str(i) for i in x])).str.split(',', expand=True).reset_index()  
    return df  

In [64]:
def main(whole_click_train_, answers_, test_qtime_):
    import time
    t0 = time.time()
    recom_item = []  
#     item_sim_list, user_item = get_sim_item(whole_click_train_, 'user_id', 'item_id', use_iif=True)  

    for i in tqdm(test_qtime_['user_id'].unique()):  
        rank_item = recommend(whole_click_train_, train_data_matrix, item_id_dict, item_id_dict_inv,  i, 500, 50)  
        for j in rank_item:  
            recom_item.append([i, j[0], j[1]])  
    # find most popular items  
    top50_click = whole_click_train_['item_id'].value_counts().index[:50].values  
    top50_click = ','.join([str(i) for i in top50_click])  

    recom_df = pd.DataFrame(recom_item, columns=['user_id', 'item_id', 'sim'])  
    result = get_predict(recom_df, 'sim', top50_click)  
    
    result1 = result.set_index('user_id')
    result1 = result1.applymap(int)
    result1['p'] = result1.values.tolist()
    print(evaluate_each_phase(result1['p'].to_dict(), answers_))
    
    t1 = time.time()
    print(f'{t1-t0}s')
    return result

In [65]:
result = main(whole_click_train, answers, test_qtime)

100%|██████████| 8462/8462 [06:45<00:00, 20.84it/s]


[0.05069723 0.02043129 0.05253582 0.02125187]
408.10114312171936s


In [ ]:
result.to_csv('/Users/luoyonggui/Downloads/baseline1_itemcf24.csv', index=False, header=None)

# 融合
简单融合，交叉后取前50个

In [20]:
dfii = pd.read_csv('baseline1.csv', header=None, index_col=0)

In [21]:
dfui = pd.read_csv('baseline4.csv', header=None, index_col=0)

In [22]:
dfui.shape

(5079, 50)

In [23]:
dfii.shape

(5079, 50)

In [24]:
dfii.head()

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
0,,,,,,,,,,,,,,,,,,,,,
1,32360,9748,95241,55738,52102,109854,16433,19228,22924,57418,...,54959,80769,19889,58929,61591,35217,90725,57001,71716,53825
2,115199,115093,113547,114017,112582,114262,110389,113345,105902,109632,...,81302,31278,54099,86488,82709,77193,4208,10612,78916,4207
11,10528,18355,12652,108373,59382,113564,34835,5977,22093,59057,...,13612,645,10800,79868,13660,15087,25635,17923,67879,5296
13,35243,60887,40179,20953,57329,54911,37662,38278,70989,78057,...,91241,41180,91503,50541,3829,52873,14289,34677,35826,64806
22,10513,43263,25435,86011,47804,56104,6969,55931,49603,115206,...,25452,10874,105485,47163,108852,113508,61850,43937,25520,6830


In [25]:
dfui.head()

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
0,,,,,,,,,,,,,,,,,,,,,
1,32360,52102,67026,90725,107027,8825,9748,44596,8632,27130,...,50410,38501,50636,13468,77683,17054,95241,38953,16420,39949
2,115199,114262,116057,113547,115093,105902,110389,113345,110740,114017,...,111447,100794,28612,115073,20439,88261,111808,109649,112542,83969
11,10528,26469,59382,82122,59057,18355,47250,34835,79868,74935,...,13612,108373,12652,41495,40879,116058,113564,26062,10800,37182
13,40179,54911,57329,20953,35243,78057,60887,38278,35945,18663,...,83445,15579,47744,37825,45839,42026,58114,8809,98083,64578
22,10513,43263,55931,25435,86011,47804,56104,6969,16511,115206,...,66736,107257,8338,34078,25520,61850,20996,376,100813,65208


In [28]:
from matplotlib.cbook import flatten

In [38]:
dfr = pd.DataFrame(columns=range(50))
for i in dfii.index.tolist():
#     print(i)
    a = dfii.loc[i].tolist()
    b = dfui.loc[i].tolist()
    dfr.loc[i] = pd.Series(flatten(zip(a, b))).drop_duplicates().iloc[:50].tolist()
#     print(pd.Series(flatten(zip(a, b))).drop_duplicates().iloc[:50])
#     break

In [39]:
dfr.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
1,32360,9748,52102,95241,67026,55738,90725,107027,109854,8825,...,73982,91290,107816,70001,36489,62185,113424,57001,58929,38885
2,115199,115093,114262,113547,116057,114017,112582,105902,110389,113345,...,4207,83410,86823,102858,115073,11848,82469,25627,81302,111447
11,10528,18355,26469,12652,59382,108373,82122,59057,113564,34835,...,13660,95346,32237,88589,39026,41221,106715,21517,31527,21941
13,35243,40179,60887,54911,57329,20953,78057,37662,38278,70989,...,90181,49054,35335,100216,58114,93449,39916,36829,69620,52999
22,10513,43263,25435,55931,86011,47804,56104,6969,49603,16511,...,30562,58916,103330,104269,46796,116155,1438,109936,25452,66736


In [40]:
dfr.shape

(5079, 50)

In [41]:
dfr.to_csv('df_merge.csv', header=None)

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)  # 设置显示数据的最大列数，防止出现省略号…，导致数据显示不全
pd.set_option('expand_frame_repr', False)  # 当列太多时不自动换行

import seaborn as sns
sns.set(font='Arial Unicode MS')  # 解决Seaborn中文显示问题
import sys
sys.path.append('/Users/luoyonggui/PycharmProjects/mayiutils_n1/mayiutils/data_prepare')
from data_explore import DataExplore as de

In [86]:
from deepctr.inputs import SparseFeat, VarLenSparseFeat
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model

from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss

# deepmatch

In [67]:
path = './data_origin/'
train_user_df = pd.read_csv(path+'underexpose_train/underexpose_user_feat.csv', names=['user_id','user_age_level','user_gender','user_city_level'])
train_user_df = train_user_df.drop_duplicates('user_id')


In [ ]:
train_item_df = pd.read_csv(path+'underexpose_train/underexpose_item_feat.csv', sep=r',\s+|,\[|\],\[',names=['item_id']+list(range(256)))
train_item_df.iloc[:, -1] = train_item_df.iloc[:, -1].str.replace(']', '').map(float)

In [129]:
whole_click_no_val = pd.read_pickle('data_gen/whole_click_no_val_set.pkl')
whole_click_val = pd.read_pickle('data_gen/whole_click_val_set.pkl')
test_qtime = pd.read_pickle('data_gen/test_qtime.pkl')

In [134]:
whole_click = pd.concat([whole_click_no_val, whole_click_val], ignore_index=True)

In [135]:
whole_click.shape, whole_click_val.shape

((480381, 3), (5079, 3))

In [139]:
whole_click.iloc[-5079:-5074]

,user_id,item_id,time
475302,22946,102303,0.983740
475303,22352,41533,0.983740
475304,20922,13603,0.983741
475305,34496,20710,0.983742
475306,15290,111807,0.983744


In [137]:
whole_click_val.head()

,user_id,item_id,time
541,22946,102303,0.983740
14900,22352,41533,0.983740
20690,20922,13603,0.983741
6518,34496,20710,0.983742
8907,15290,111807,0.983744


In [217]:
data = pd.merge(pd.merge(whole_click, train_user_df, how='left'), train_item_df, how='left')

In [218]:
data.loc[:, 'user_age_level'] = data.loc[:, 'user_age_level'].fillna(0)

data.loc[:, 'user_city_level'] = data.loc[:, 'user_city_level'].fillna(0)

data.loc[:, 'user_gender'] = data.loc[:, 'user_gender'].fillna('<UNK>')

data.head()

,user_id,item_id,time,user_age_level,user_gender,user_city_level,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,...,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255
0,25129,3852,0.98374,0.0,<UNK>,0.0,2.571915,-1.814055,3.279824,-2.754879,-2.226882,0.628462,-0.960409,1.276046,-3.626350,-2.434744,0.246905,-0.608179,5.237006,-0.237802,-0.199239,0.064605,-2.160981,0.971480,-2.408688,-0.884277,1.025688,-1.907494,-1.067752,-2.969791,-0.127235,0.711640,2.428432,-0.760012,0.370263,1.928983,-0.747229,-1.465866,-1.025934,-3.069760,2.918849,-2.058115,-0.805159,-2.803984,2.498572,3.038785,-1.862499,0.825423,-0.043094,3.431325,...,2.421870,0.667413,-2.090987,-1.924392,-0.108686,-2.947167,-2.780642,0.503893,1.921814,-1.270335,-0.167638,-2.014415,0.980605,-0.432822,-1.538932,-0.497246,3.262029,-1.889152,2.447979,2.023919,1.444185,-0.095509,1.549015,3.191395,-0.198340,-0.035811,-0.905403,0.748054,1.069559,0.216951,-1.438549,-2.388932,2.662791,4.099761,-0.267581,-3.321726,-0.155415,-0.525411,1.735601,-0.589674,1.453013,1.571093,-1.223752,-4.705770,-2.396826,0.079413,0.606347,1.943259,1.899397,-1.245537
1,20937,18599,0.98374,0.0,<UNK>,0.0,2.869179,-3.099122,2.084314,-2.454703,-2.094862,-0.662508,-3.808012,1.813844,-6.038182,-0.180724,-1.103885,-1.550228,5.279609,-0.503311,3.120579,0.063907,0.029678,0.480394,-1.509667,0.377320,1.514299,-1.014878,1.615282,-5.186666,-0.365947,-0.587060,0.525926,0.869364,0.476346,1.914612,-4.939361,-3.769417,2.798874,1.344090,3.873302,-3.572078,-1.601236,-3.016315,2.205385,3.553573,-2.045253,-0.157650,2.266847,5.036000,...,0.481929,-0.373578,0.544273,0.111372,-1.719494,-1.100947,-2.666645,-3.169693,-0.838876,-0.808355,3.324217,3.368498,0.751686,-2.285471,-1.774423,-1.140164,3.305172,-4.562526,-0.137779,0.451743,2.721603,2.801044,3.219163,1.965619,0.347214,-1.566591,2.519880,1.113690,0.224589,0.269521,-1.206637,-3.876595,2.657075,0.765382,0.361337,-2.198026,0.345006,3.772189,1.842122,-0.739535,3.972774,-0.904352,3.413301,-2.395801,-0.955557,3.624955,1.230665,1.725096,-1.393228,-4.908030
2,22528,48057,0.98374,0.0,<UNK>,0.0,4.078118,-0.378743,0.272353,-2.633809,-1.767918,0.949223,1.033278,0.314563,-4.805015,-4.950263,2.126089,1.625062,4.022007,-2.070587,0.192334,-0.201567,-1.964089,0.027071,-2.421269,-1.420727,-2.351256,-1.689599,-0.386515,-1.401658,-1.278733,0.355593,1.742722,-0.018574,-1.864175,0.927499,0.648029,-2.921231,-1.621955,0.473017,2.966018,-1.656037,0.981539,-3.516828,2.144614,2.134675,-2.026774,-0.093928,2.424305,-0.054545,...,-0.120378,-3.074750,-0.436548,1.226916,2.349869,2.804942,-0.416353,0.158974,2.758401,-1.814317,0.509540,2.357375,1.151527,-0.204829,-2.014134,-1.505196,0.490951,-0.478856,4.660158,1.820699,1.378949,-0.040161,0.497809,4.419123,0.049237,3.057560,3.268921,1.871532,1.794398,2.954940,-0.558485,0.106806,0.409342,0.381947,2.422366,0.706619,-5.211069,1.736306,3.246878,-0.308061,-1.036637,2.311875,-0.909021,-3.420048,2.800025,1.820050,0.257765,2.410485,1.843264,0.536979
3,12136,28195,0.98374,7.0,F,4.0,4.158704,-2.213841,3.299623,-3.816672,1.817951,-0.562453,-3.637601,-0.311089,-5.418423,-2.103671,-0.442526,-3.839618,4.380653,-3.727894,1.042084,-2.424195,3.287570,-0.606853,-1.033391,0.160277,-0.038618,-1.071147,0.780368,-0.159890,-1.548846,-2.328499,-0.205781,-3.355978,-5.415924,-1.503188,-5.305884,-1.843716,-0.245827,3.008088,3.399326,-3.194445,-3.397100,-0.312699,2.398918,4.553303,-2.690272,1.037717,1.235074,4.735878,...,-1.572505,-1.981386,-1.908951,3.034426,-4.374824,0.489178,-3.051432,-2.832758,-2.986957,-1.558040,4.542650,-0.078699,2.846123,2.866818,-2.151008,0.465624,3.206414,-2.886617,-0.973224,4.858570,0.401090,1.429223,-1.306388,2.436626,0.375304,0.796462,-1.397474,0.019915,-4.959344,-0.943585,0.348210,-2.549550,1.375585,-3.331098,0.260209,-1.604760,4.503458,0.

In [142]:
data.shape

(480381, 262)

## 构建特征列，训练模型，导出embedding

In [222]:
sparse_features = ["item_id", 
                   "user_id","user_gender", "user_age_level", "user_city_level", ]


# 1.Label Encoding for sparse features,and process sequence features with `gen_date_set` and `gen_model_input`

features = ["item_id", "user_id",
                    "user_gender", "user_age_level", "user_city_level", ]
feature_max_idx = {}
lbe_list = []
for feature in features:
#     print(feature)
    lbe = LabelEncoder()
#     data.loc[:, feature] = data[feature].fillna('<UNK>')
    data[feature] = lbe.fit_transform(data[feature]) + 1
#     data.loc[data[feature].notnull(), feature] = lbe.fit_transform(data.loc[data[feature].notnull(), feature]) + 1
    feature_max_idx[feature] = data[feature].max() + 1
    lbe_list.append(lbe)

In [223]:
user_profile = data[["user_id", "user_gender", "user_age_level", "user_city_level"]].drop_duplicates('user_id')

item_profile = data[["item_id"]].drop_duplicates('item_id')

user_profile.set_index("user_id", inplace=True)

In [268]:
user_item_list = data.groupby("user_id")['item_id'].apply(list)
# User点击的item list的平均长度为20
user_item_list.map(len).describe()

count    23816.000000
mean        20.170516
std         19.394592
min          2.000000
25%          8.000000
50%         14.000000
75%         25.000000
max        233.000000
Name: item_id, dtype: float64

In [269]:
user_item_list.head(2)

user_id
1    [42465, 14686, 48588, 41442, 47502, 42615, 346...
2    [39649, 4326, 15450, 39742, 59988, 56439, 3953...
Name: item_id, dtype: object

In [221]:
SEQ_LEN = 25 # 75%分位点
negsample = 1 # 产生一倍的负样本

In [225]:
import random
import numpy as np
from tqdm import tqdm
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

def gen_data_set(data, negsample: int=0):
    """
    negsample: 负样本数相对于正样本数的倍数
    """
    data.sort_values("time", inplace=True)
    item_ids = data['item_id'].unique()

    train_set = []
    val_set = []
    test_set = []
    pred_user_list = lbe_list[1].transform(whole_click_val.user_id)+1
    print(len(pred_user_list))
#     print(data.iloc[(-whole_click_val.shape[0]):].user_id.nunique())
    for reviewerID, hist in tqdm(data.groupby('user_id')):
#         user_id = hist.user_id.iloc[0]
        pos_list = hist['item_id'].tolist()
        rating_list = [1] * len(hist['item_id'])

        if negsample > 0:
            # 产生负样本，策略：从用户没有评分的items中随机有放回采样
            candidate_set = list(set(item_ids) - set(pos_list))
            neg_list = np.random.choice(candidate_set,size=len(pos_list)*negsample,replace=True)
        for i in range(1, len(pos_list)):
            hist = pos_list[:i]
            if i != len(pos_list) - 1:
                train_set.append((reviewerID, hist[::-1], pos_list[i], 1,len(hist[::-1]),rating_list[i]))
                for negi in range(negsample):
                    train_set.append((reviewerID, hist[::-1], neg_list[i*negsample+negi], 0,len(hist[::-1])))
            else:
                # 取最后一个点击过的item 作为 val set
                val_set.append((reviewerID, hist[::-1], pos_list[i],1,len(hist[::-1]),rating_list[i]))
        if reviewerID in pred_user_list:
            test_set.append((reviewerID, pos_list[::-1], -1,1,len(pos_list[::-1]),1))
    random.shuffle(train_set)
    random.shuffle(val_set)

    print(len(train_set[0]),len(test_set[0]))

#     return test_set
    return train_set,val_set,test_set

In [226]:
# pred_set = gen_data_set(data, negsample)

5079


100%|██████████| 23816/23816 [00:07<00:00, 3099.13it/s]


In [148]:
train_set, test_set, pred_set = gen_data_set(data, negsample)

100%|██████████| 23816/23816 [07:30<00:00, 52.83it/s]


6 6


In [150]:
def gen_model_input(train_set,user_profile,seq_max_len):

    train_uid = np.array([line[0] for line in train_set])
    train_seq = [line[1] for line in train_set]
    train_iid = np.array([line[2] for line in train_set])
    train_label = np.array([line[3] for line in train_set])
    train_hist_len = np.array([line[4] for line in train_set])

    train_seq_pad = pad_sequences(train_seq, maxlen=seq_max_len, padding='post', truncating='post', value=0)
    train_model_input = {"user_id": train_uid, "item_id": train_iid, "hist_item_id": train_seq_pad,
                         "hist_len": train_hist_len}
    
#     no_feat_users = pd.Series(train_model_input['user_id']).drop_duplicates()
#     no_feat_users = no_feat_users[~no_feat_users.isin(user_profile.index.tolist())]
#     for i in no_feat_users:
#         user_profile.loc[i, ["user_gender", "user_age_level", "user_city_level"]] = [0, 0, 0]
    for key in ["user_gender", "user_age_level", "user_city_level"]:
        
        train_model_input[key] = user_profile.loc[train_model_input['user_id'], key].values

    return train_model_input,train_label

In [151]:
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)
pred_model_input, _ = gen_model_input(pred_set, user_profile, SEQ_LEN)

In [227]:
# pred_model_input, _ = gen_model_input(pred_set, user_profile, SEQ_LEN)

In [152]:
train_model_input

{'user_id': array([  421, 14143, 11801, ...,  5040,  6868,  1884]),
 'item_id': array([15534, 32906, 14885, ..., 52854,  8216, 42627]),
 'hist_item_id': array([[48986, 38326,  6232, ..., 12398,  4768, 22180],
        [21534, 56553, 53714, ...,     0,     0,     0],
        [ 2359,  2804,  2337, ...,     0,     0,     0],
        ...,
        [ 5453, 29545, 45489, ...,  9160, 28743,     0],
        [32971, 17707, 43940, ...,     0,     0,     0],
        [39596, 13682, 21823, ...,     0,     0,     0]], dtype=int32),
 'hist_len': array([30, 22, 15, ..., 24,  3,  3]),
 'user_gender': array([1, 2, 1, ..., 1, 1, 2]),
 'user_age_level': array([1, 5, 1, ..., 1, 1, 7]),
 'user_city_level': array([1, 2, 1, ..., 1, 1, 4])}

In [229]:
pred_model_input['user_id'].shape

(5079,)

In [154]:
pd.Series(train_label).value_counts()

1    432749
0    432749
dtype: int64

In [155]:
# 2.count #unique features for each sparse field and generate feature config for sequence feature

embedding_dim = 32

user_feature_columns = [SparseFeat('user_id', feature_max_idx['user_id'], 16),
                        SparseFeat("user_gender", feature_max_idx['user_gender'], 16),
                        SparseFeat("user_age_level", feature_max_idx['user_age_level'], 16),
                        SparseFeat("user_city_level", feature_max_idx['user_city_level'], 16),
                        VarLenSparseFeat(SparseFeat('hist_item_id', feature_max_idx['item_id'], embedding_dim,
                                                    embedding_name="item_id"), SEQ_LEN, 'mean', 'hist_len'),
                        ]

item_feature_columns = [SparseFeat('item_id', feature_max_idx['item_id'], embedding_dim)]

In [156]:
# 3.Define Model and train

K.set_learning_phase(True)

model = YoutubeDNN(user_feature_columns, item_feature_columns, num_sampled=256, user_dnn_hidden_units=(128,64, embedding_dim))
# model = MIND(user_feature_columns,item_feature_columns,dynamic_k=True,p=1,k_max=2,num_sampled=5,user_dnn_hidden_units=(64,16),init_std=0.001)

model.compile(optimizer="adam", loss=sampledsoftmaxloss)  # "binary_crossentropy")

## train

In [290]:
history = model.fit(train_model_input, train_label,  # train_label,
                    batch_size=512, epochs=200, verbose=1, validation_split=0.02, )

Train on 848188 samples, validate on 17310 samples
Epoch 1/200
848188/848188 [==============================] - 104s 122us/sample - loss: 3.4546 - val_loss: 5.5275
Epoch 2/200
848188/848188 [==============================] - 78s 92us/sample - loss: 3.4459 - val_loss: 5.5171
Epoch 3/200
848188/848188 [==============================] - 79s 93us/sample - loss: 3.4442 - val_loss: 5.4468
Epoch 4/200
848188/848188 [==============================] - 78s 92us/sample - loss: 3.4293 - val_loss: 5.5343
Epoch 5/200
848188/848188 [==============================] - 80s 94us/sample - loss: 3.4341 - val_loss: 5.6399
Epoch 6/200
848188/848188 [==============================] - 79s 93us/sample - loss: 3.4279 - val_loss: 5.5931
Epoch 7/200
848188/848188 [==============================] - 80s 94us/sample - loss: 3.4255 - val_loss: 5.5747
Epoch 8/200
848188/848188 [==============================] - 79s 93us/sample - loss: 3.4193 - val_loss: 5.6032
Epoch 9/200
848188/848188 [==============================] 

848188/848188 [==============================] - 82s 97us/sample - loss: 3.3391 - val_loss: 6.4637
Epoch 146/200
848188/848188 [==============================] - 83s 97us/sample - loss: 3.3422 - val_loss: 6.6553
Epoch 147/200
848188/848188 [==============================] - 83s 97us/sample - loss: 3.3400 - val_loss: 6.4908
Epoch 148/200
848188/848188 [==============================] - 84s 99us/sample - loss: 3.3381 - val_loss: 6.5810
Epoch 149/200
848188/848188 [==============================] - 82s 97us/sample - loss: 3.3353 - val_loss: 6.5716
Epoch 150/200
848188/848188 [==============================] - 83s 98us/sample - loss: 3.3352 - val_loss: 6.6701
Epoch 151/200
848188/848188 [==============================] - 83s 98us/sample - loss: 3.3466 - val_loss: 6.6318
Epoch 152/200
848188/848188 [==============================] - 83s 98us/sample - loss: 3.3409 - val_loss: 6.6206
Epoch 153/200
848188/848188 [==============================] - 83s 97us/sample - loss: 3.3440 - val_loss: 6.63

In [97]:
type(Model)

tensorflow.python.keras.engine.training.Model

## val

In [291]:
# 4. Generate user features for testing and full item features for retrieval
test_user_model_input = test_model_input
all_item_model_input = {"item_id": item_profile['item_id'].values,}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)



test_true_label = {line[0]:[line[2]] for line in test_set}

import numpy as np
import faiss
from tqdm import tqdm
from deepmatch.utils import recall_N

index = faiss.IndexFlatIP(embedding_dim)
# faiss.normalize_L2(item_embs)
index.add(item_embs)

# faiss.normalize_L2(user_embs)
D, I = index.search(user_embs, 50)

I.shape, D.shape

D

s = []
hit = 0
for i, uid in tqdm(enumerate(test_user_model_input['user_id'])):
#     try:
    if True:
        # 获取预测的item_id
        pred = [item_profile['item_id'].values[x] for x in I[i]]
#         print(pred)
#         break
        filter_item = None
        recall_score = recall_N(test_true_label[uid], pred, N=50)
        s.append(recall_score)
        if test_true_label[uid] in pred:
            hit += 1
#     except:
#         print(i)
print("")
print("recall", np.mean(s))
print("hit rate", hit / len(test_user_model_input['user_id']))

(23816, 32)
(61905, 32)


23816it [00:05, 4102.24it/s]


recall 0.016753443063486733
hit rate 0.016753443063486733


In [295]:
set([1, 0, 1])

{0, 1}

## pred

In [292]:
# 4. Generate user features for testing and full item features for retrieval
# test_user_model_input = test_model_input
test_user_model_input = pred_model_input
all_item_model_input = {"item_id": item_profile['item_id'].values,}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)



index = faiss.IndexFlatIP(embedding_dim)
# faiss.normalize_L2(item_embs)
index.add(item_embs)

# faiss.normalize_L2(user_embs)
# D 距离矩阵， I 索引矩阵
D, I = index.search(user_embs, 500)

I.shape, D.shape



dfr = pd.DataFrame(pd.DataFrame(columns=[f'item_{i}' for i in range(50)]))


s = []
hit = 0

# lbe_list[1].inverse_transform([i])
u1 = test_user_model_input['user_id']
for i, uid in tqdm(enumerate(lbe_list[1].inverse_transform(test_user_model_input['user_id']-1))):
#     try:
    if True:
        
        
        # 获取预测的item_id
        pred = [item_profile['item_id'].values[x] for x in I[i]]
        dfr.loc[uid] = lbe_list[0].inverse_transform([ii-1 for ii in pred if ii not in user_item_list.loc[u1[i]]])[:50]
        
# print(dfr.head())

(5079, 32)
(61905, 32)


5079it [01:25, 59.15it/s]


In [293]:
dfr.to_csv('/Users/luoyonggui/Downloads/baseline1_itemcf6.csv', header=None)

In [284]:
dfr.head()

,item_0,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,item_10,item_11,item_12,item_13,item_14,item_15,item_16,item_17,item_18,item_19,item_20,item_21,item_22,item_23,item_24,item_25,item_26,item_27,item_28,item_29,item_30,item_31,item_32,item_33,item_34,item_35,item_36,item_37,item_38,item_39,item_40,item_41,item_42,item_43,item_44,item_45,item_46,item_47,item_48,item_49
1,108007,102958,84267,94926,98806,108288,93960,105496,30956,76595,105886,64109,109322,104436,74204,63609,108151,109416,109735,75446,102650,99058,83204,80776,61628,114125,112806,87570,105116,94961,111560,88947,95690,112555,113094,79754,105590,99470,47192,93538,90602,80555,113708,93116,72782,102274,96063,95743,66688,96481
2,102958,94926,108007,98806,93960,109322,76595,94961,105496,109416,104436,96063,74204,105116,113115,84267,30956,111318,113361,114125,113708,95965,99058,63609,111560,111751,93538,88947,112959,112806,73835,104747,105886,113467,75326,112555,108151,80555,55823,81311,108010,105590,110033,94632,104880,56287,104900,75446,102650,90602
11,112351,88002,102863,98129,82194,110355,69793,53751,103411,113976,108948,25095,115250,116130,78213,106023,82044,39704,109136,53888,106286,102399,111215,48349,89888,104693,82138,52766,69911,105224,100821,88528,93379,96137,48051,19758,92218,110903,32990,28014,114125,101938,116778,94797,41138,80976,85899,101321,116697,108782
13,108324,101669,102924,84166,103279,97032,111751,105886,108938,103630,81764,105423,45157,104693,84267,111599,74906,108567,78086,114495,57964,62216,88242,113655,54437,82515,87541,83457,87061,103142,115059,85254,100215,114085,69793,110077,91715,41170,87710,66873,72203,90813,76215,82904,73584,99030,82737,102129,62151,88528
22,108324,78086,97032,57964,103279,84166,101669,96910,88242,103630,102924,111599,100215,81764,105886,62216,111751,105423,113567,84267,82737,104693,108567,82138,73584,54437,103142,85254,74906,87061,90217,72203,79932,105411,109136,82850,45157,85564,82419,109182,108938,91715,44185,113655,82515,78229,82109,52268,62151,83661


In [263]:
dfr.index

Int64Index([    1,     2,    11,    13,    22,    34,    35,    44,    45,
               55,
            ...
            35345, 35354, 35355, 35365, 35366, 35388, 35389, 35398, 35421,
            35422],
           dtype='int64', length=5079)

In [265]:
whole_click_val.user_id.sort_values()

7848         1
3756         2
15374       11
16425       13
1168        22
         ...  
11625    35388
14663    35389
13316    35398
16941    35421
16203    35422
Name: user_id, Length: 5079, dtype: int64

In [262]:
print(dfr.head(2))

   item_0  item_1  item_2  item_3  item_4  item_5  item_6  item_7  item_8  item_9 item_10 item_11 item_12 item_13 item_14 item_15 item_16 item_17 item_18 item_19 item_20 item_21 item_22 item_23 item_24 item_25 item_26 item_27 item_28 item_29 item_30 item_31 item_32 item_33 item_34 item_35 item_36 item_37 item_38 item_39 item_40 item_41 item_42 item_43 item_44 item_45 item_46 item_47 item_48 item_49
1  102958   94961  113361  105886  105496  105116  111560   88947  113115  109322   89975   45530  114125   74204  113094   81353   99058   93960  113708  107900  105012  108007   88936  113467  108621   91838  112562  104818   73689   84267   77012  109416   89082   95171   81715  104436   95965  102263   95743  110740   47192   52995  101509   90726   88931  104731  103740   98370   59702   97400
2   94961  102958  113361  113115  105116  113467  112562  111560   88947  109322  105496  114125  108621  113708  112959  116114  103605  109121  110838  101975  115874   95965   81353   74204  1

In [259]:

dfr = dfr.applymap(lambda: lbe_list[0].inverse_transform)
print(dfr.head())

ValueError: bad input shape ()